In [ ]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torchvision
import cv2
import torch.nn.functional as F 
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms  as transforms
import math
from earlyStopping import EarlyStopping

In [ ]:
earlyStop = EarlyStopping()

In [ ]:
BATCH_SIZE=2

In [ ]:
# Selecting the CUDA:1 as for the training
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [ ]:
# In case os does not allow us to load the files
# os.listdir('/media/user/New Volume/AliveDataset/ISIC_Datasets/SIIM-ISIC-2020/')

In [ ]:
csv_path = '/media/user/New Volume/AliveDataset/ISIC_Datasets/SIIM-ISIC-2020/train.csv'
IMG_DIR = '/media/user/New Volume/AliveDataset/ISIC_Datasets/SIIM-ISIC-2020/train'

In [ ]:
df = pd.read_csv(csv_path)
df.head()

In [ ]:
# checking the sum of zero that contain in each column
df.isnull().sum()

In [ ]:
# Map male and female categorical variable into 0 and 1
df['sex'] = df['sex'].map({'male':1,'female':0})

# Fill All NaN values with -1
df['sex'] = df['sex'].fillna(-1)

In [ ]:
# Remove the all data that has no value for the sex
df = df.query('sex >=0')

In [ ]:
# Map all anotomical location to numeric values which starts from 0
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].map({'head/neck':0, 'upper extremity':1, 'lower extremity':2, 'torso':3,
       'palms/soles':4, 'oral/genital':5})



# Unknown Location will be filled with 6
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].fillna(6)

In [ ]:
# Fill Nan Age values with 0
df['age_approx'] = df['age_approx'].fillna(0)

In [ ]:
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

labels = [0, 1, 2, 3, 4, 5, 6, 7, 8]


# Age bininig each age of patient
df['age_bin']  = pd.cut(df['age_approx'],bins=bins,labels=labels,right=True,include_lowest=True)

In [ ]:
df.isnull().sum()

In [ ]:
#
location_vocab_size = len(df['anatom_site_general_challenge'].unique())

print(location_vocab_size)

In [ ]:
# Number of Genders
sex_vocab_size = len(df['sex'].unique())

print(sex_vocab_size)

In [ ]:
# Number of Age Bins
age_vocab_size = len(df['age_bin'].unique())
print(age_vocab_size)

In [ ]:
transformations  = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485,0.456,0.406],
                                                            std=[0.229,0.224,0.225])])

In [ ]:
class SIIMDataAcquisition(Dataset):
    def __init__(self,df,transforms=None):
        super().__init__()
        self.df = df
        
        self.transforms = transforms
    
    def __len__(self):
        return len(self.df['patient_id'].unique())
    
    def __getitem__(self, idx):

        per_patient = []
        patient_id = self.df['patient_id'].unique()[idx]
        filtered_df = self.df.loc[self.df['patient_id'].eq(patient_id)].sort_values('age_bin')
        sex = torch.FloatTensor(filtered_df['sex'].to_list())
        age = torch.FloatTensor(filtered_df['age_bin'].to_list())
        location = torch.FloatTensor(filtered_df['anatom_site_general_challenge'].to_list())
        lesion = filtered_df['image_name'].to_list()

        labels = torch.FloatTensor(filtered_df['target'].to_list())


        for image_id in lesion:
            image_path = os.path.join(IMG_DIR,image_id +'.jpg')

            image = Image.open(image_path)

            image = self.transforms(image)
            per_patient.append(image)

        
        concat_features = torch.stack(per_patient)

       


        return concat_features,sex.T,age.T,location.T,labels
           


In [ ]:
def dynamic_collate_fn(batch):

    s_l = []
    gender = []
    lo = []
    age_array = []
    gt_array = []
    
    for data in batch:
        skin_lesion = data[0]
        sex = data[1]
        age = data[2]
        locations = data[3]
        labels = data[4]

        s_l.append(skin_lesion)
        gender.append(sex)
        lo.append(locations)
        age_array.append(age)
        gt_array.append(labels)



    return {'skin_lesions':s_l,
            'gender':gender,
            'age':age_array,
            'locations':lo,
            'gt_label':gt_array
            }


In [ ]:
# train and validation split into ratio of 8:2

train_df = df.sample(frac=0.8,random_state=200)
val_df   = df.drop(train_df.index)

In [ ]:

train_dataset = SIIMDataAcquisition(df=train_df,transforms=transformations)
train_dataLoader = torch.utils.data.DataLoader(dataset=train_dataset ,batch_size=BATCH_SIZE,
                                               collate_fn=dynamic_collate_fn,shuffle=True)

In [ ]:

val_dataset = SIIMDataAcquisition(df=val_df,transforms=transformations)
val_dataLoader = torch.utils.data.DataLoader(dataset=val_dataset  ,batch_size=BATCH_SIZE,
                                             collate_fn=dynamic_collate_fn,shuffle=False)


In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=120):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(size=(max_len, d_model))
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        
        # print(x.size()) [64, 7]
        
        x = x + self.pe[:,:x.size(1)] 
        return self.dropout(x)

In [ ]:
class FeatureExtractionResnet(nn.Module):

    # Model is trained on SIIM ISIC 2020 in binary classification setting
    TrainedModel = torch.load('/home/user/Documents/Alive/Alive_Repo/BaseLineModels/IRSoftAttention/finetunedResnet.pt').to(device)

    TrainedModel.eval()
    
    def __init__(self,hidden_dimension,num_ftrs:int=2048):
        super().__init__()

        FeatureExtractionResnet.TrainedModel.fc = nn.Identity()

        self.linear_projection = nn.Linear(num_ftrs,hidden_dimension)

    def forward(self,x):
        x = FeatureExtractionResnet.TrainedModel(x)
        return self.linear_projection(x)

In [ ]:
class InputEmbedding(nn.Module):
    def __init__(self,vocab_size,hidden_dim):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size,hidden_dim)

    def forward(self,x):
        return self.embedding(x)

In [ ]:
class EmbeddingModule(nn.Module):
    def __init__(self,sex_vocab,age_vocab,location_vocab,max_seq_length,hidden_dim,batch_size):
        super().__init__()


        self.hidden_dim = hidden_dim
        
        self.sex_embedding = InputEmbedding(sex_vocab,self.hidden_dim)
        self.age_embedding = InputEmbedding(age_vocab,self.hidden_dim)
        self.location_embedding = InputEmbedding(location_vocab,self.hidden_dim)
        
        self.feature_extraction = FeatureExtractionResnet(self.hidden_dim)
        self.type_embedding_matrix = nn.Parameter(torch.randn(batch_size,self.hidden_dim,3))
        self.positional_encoding = PositionalEncoding(self.hidden_dim)

        self.max_seq_lenth = max_seq_length

    def forward(self,age,sex,locations,lesions):


    
        # # Getting Batch Size
        # batch_size = len(sex)


        


        # # Rearrange Type Embedding from 1,64,3 -> Batch_Size,64,3
        # self.type_embedding_matrix = nn.Parameter(einops.repeat(self.type_embedding_matrix,
        #                                            'b h c -> (repeat b) h c',repeat=batch_size))
        
        # Getting Sex Embedding
        sex_emb = [self.sex_embedding(s.long()).T for s in sex]



        # Lesion Feature Extraction from PreTrained ResNet101 on SIIM ISIC 2020
        feat_ext = [self.feature_extraction(ls).T for ls in lesions]
        

        # Anatomic Site Embedding
        anatomic_site_emb = [self.location_embedding(lc.long()).T for lc  in locations]


        
        # Age Embedding
        age_emb = [self.age_embedding(ag.long()).T for ag in age]

        
        


        # Adding Lesion Extraction and Anatomical Location Together
        Qp_per_batch = [x+y for x,y in zip(feat_ext,anatomic_site_emb)]
        

        

        

        # Concatenating parameter token with age embedding(+positional encoding) for each sample in the batch 
        S_P = [F.pad(self.positional_encoding(torch.cat([self.type_embedding_matrix[idx,:,0].unsqueeze(1),val],dim=1)),(0, self.max_seq_lenth-val.shape[1])) 
               for idx,val in enumerate(age_emb)]

       

        # Concatenating parameter token with gender embeddingfor each sample in the batch 
        Y_P = [F.pad(torch.cat([self.type_embedding_matrix[idx,:,1].unsqueeze(1),val],dim=1),(0, self.max_seq_lenth-val.shape[1])) 
               for idx,val in enumerate(sex_emb)]
        


        # Concatenating parameter token with lesion+location embedding for each sample in the batch 
        Q_p = [F.pad(torch.cat([self.type_embedding_matrix[idx,:,2].unsqueeze(1),val],dim=1),(0, self.max_seq_lenth-val.shape[1])) 
               for idx,val in enumerate(Qp_per_batch)]
        

       
     
       
        # Combined all embeddings and form a batch 
        combined_embedding = torch.stack([torch.cat([x,y,z],dim=-1) for x,y,z in zip(Q_p,Y_P,S_P)])

       
       


        return combined_embedding

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self,hidden_dimenion,num_heads,dropout):

        super().__init__()
        self.hidden_dimension = hidden_dimenion
        self.num_heads = num_heads
        self.dropout = dropout


        self.norm = nn.LayerNorm(self.hidden_dimension)

        self.multihead = nn.MultiheadAttention(self.hidden_dimension,
                                               self.num_heads,self.dropout)
        
        self.enc_mlp = nn.Sequential(
            nn.Linear(self.hidden_dimension,self.hidden_dimension*4),
            nn.GELU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dimension*4,self.hidden_dimension),
            nn.GELU()
        )

    def forward(self,embeddings,masks):
        firstnorm = self.norm(embeddings)
        attention_out = self.multihead(firstnorm,firstnorm,firstnorm,key_padding_mask=masks)[0]

        residual_1 = attention_out + firstnorm

        secondnorm = self.norm(residual_1)

        ff_out = self.enc_mlp(secondnorm)

        return ff_out + residual_1

In [ ]:
def encoderMasks(sequence_length,batch_size,concat_feature):

    masks = torch.zeros(batch_size,concat_feature)

    for batch_idx in range(batch_size):
        for seq_idx in sequence_length:
            for xi in range(seq_idx):
                masks[batch_idx][xi] = 1



    return masks.T.to(device)

In [ ]:
class TransFormerModel(nn.Module):
    def __init__(self,num_layers:int,num_heads:int,hidden_dim:int,
                 num_classes:int,dropout:float,max_lesion_length:int=115):
        super().__init__()


        self.hidden_dim = hidden_dim

        self.max_lesion_length = max_lesion_length
        self.embedding = EmbeddingModule(sex_vocab_size,
                                         age_vocab_size,
                                         location_vocab_size,
                                         self.max_lesion_length,
                                         self.hidden_dim ,
                                         BATCH_SIZE)
        

        
        
        self.encodeLayers = [EncoderBlock(self.hidden_dim ,num_heads,0.1).to(device) for i in range(num_layers)]


        self.MLP_head = nn.Sequential(
            nn.LayerNorm(self.hidden_dim*3),
            nn.Linear(self.hidden_dim *3 ,self.hidden_dim ),
            nn.Linear(self.hidden_dim,num_classes)
        )

        self.activation = nn.Sigmoid()

    def forward(self,age,sex,locations,lesions):

        
        encode_output = self.embedding(age,sex,locations,lesions).transpose(1,2) # [4, 348, 64]


        seq_length = [j.size()[0] for j in age]
        
        KeyPaddingMasks = encoderMasks(seq_length, encode_output.size()[0],encode_output.size()[1] )

       


        for encode_layer in self.encodeLayers:
            encode_output = encode_layer(encode_output,KeyPaddingMasks)


        # Getting class token from each batch and Flatten the last two dimension
        class_tokens = encode_output[:,[0,115,231],:].view(-1,3*self.hidden_dim)

        

        # print('this has been completed untill this point')
        return self.activation(self.MLP_head(class_tokens))

        

In [ ]:
# Pre-Defined Parameters
dimension_of_projection = 64
classification_dim = 2048
max_lesion_length = max(df.groupby('patient_id')['image_name'].count())
min_lesion_length = min(df.groupby('patient_id')['image_name'].count())
num_layers = 4
num_heads = 4
num_classes=1

In [ ]:
model = TransFormerModel(num_layers=num_layers,num_heads=num_heads,
                         hidden_dim=dimension_of_projection,
                         num_classes=num_classes,
                         dropout=0.1,
                         max_lesion_length=max_lesion_length).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=10e-3,weight_decay=0.03)
criterion = nn.BCELoss()
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer)

In [ ]:
def getMaxOfLabels(labels):
    data =[]
    for i in labels:
        data.append(torch.max(i))
    return torch.stack(data)

In [ ]:
def trainEpoch(dataloader,model,criteria,optimizer):

    model.train()
    train_losses = []

    for batch_idx,(data) in enumerate(dataloader):

        age = [x.to(device) for x in data['age']]
        gender = [x.to(device) for x in data['gender']]
        anatomic_location  = [x.to(device) for x in data['locations']]
        skin_lesions = [x.to(device) for x in data['skin_lesions']]
        labels = [x.to(device) for x in data['gt_label']]

        refined_labels = getMaxOfLabels(labels).view(-1,1)


        optimizer.zero_grad()

        output = model(age,gender,anatomic_location,skin_lesions)

        loss = criteria(output,refined_labels)

        loss.backward()

        optimizer.step()

        if batch_idx%10 ==0:
            print(f'Training loss at Batch {batch_idx} --> {loss.item()}')

        train_losses.append(loss.item())

    return train_losses

In [ ]:
def valEpoch(dataloader,model,criteria):

    model.eval()
    val_losses = []

    for batch_idx,(data) in enumerate(dataloader):

        age = [x.to(device) for x in data['age']]
        gender = [x.to(device) for x in data['gender']]
        anatomic_location  = [x.to(device) for x in data['locations']]
        skin_lesions = [x.to(device) for x in data['skin_lesions']]
        labels = [x.to(device) for x in data['gt_label']]

        refined_labels = getMaxOfLabels(labels).view(-1,1)


        output = model(age,gender,anatomic_location,skin_lesions)

        loss = criteria(output,refined_labels)

        

        if batch_idx%10 ==0:
            print(f'Validation loss at Batch {batch_idx} --> {loss.item()}')

        val_losses.append(loss.item())

    return val_losses

In [ ]:
def main(model,trainLoader,ValLoader,criteria,optimizer,scheduler,epochs):

    train_losses = []
    val_losses   = []
    for epoch in range(epochs):
        train_loss = trainEpoch(trainLoader,model,criteria,optimizer)
        val_loss = valEpoch(ValLoader,model,criteria)


        avg_train_loss = np.average(train_loss)
        avg_val_loss = np.average(val_loss)


        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)

        earlyStop(avg_val_loss,model)

        scheduler.step()

        if earlyStop.early_stop:
            print('Early Stopping')
            break

    model.load_state_dict(torch.load('checkpoint.pt'))


    return model,train_losses,val_losses

        

In [ ]:
SavedModel,trainLosses,ValLosses = main(model,train_dataLoader,val_dataLoader,criterion,optimizer,scheduler,100)